Вычислительная математика, практикум 1.
Линейные системы и метод наименьших квадратов с помощью LU-разложения

In [2]:
import numpy as np

def LU_decompose(input: np.array):
    n = input.shape[0]
    U = input
    L = np.eye(n, dtype=input.dtype)

    for k in range(n - 1):
        L[k+1:, k] = U[k+1:, k] / U[k, k]
        U[k+1:, k:] -= np.expand_dims(L[k + 1:, k], 1) @ np.expand_dims(U[k, k:], 0)

    return L, U

In [3]:
def solve_system(A, b):
    L, U = LU_decompose(A)
    n = A.shape[0]
    
    def solve_upper(A, b):
        x = np.zeros(n)
        for i in range(n):
            x[i] = b[i] / A[i, i]
            b[i + 1:] -= A[i + 1:, i] * x[i]
        return x

    def solve_lower(A, b):
        x = np.zeros(n)
        for i in range(n - 1, -1, -1):
            x[i] = b[i] / U[i, i]
            b[:i] -= A[:i, i] * x[i]
        return x

    y = solve_upper(L, b)
    x = solve_lower(U, y)
    return x

In [4]:
def solve_lstsq(A, b):
    L, U = LU_decompose(A.T @ A)
    y = solve_system(L, A.T @ b)
    x = solve_system(U, y)
    return x

In [5]:
eps = 1e-4
for i in range(1, 100):
    A = np.random.randn(i, i)
    while (np.linalg.matrix_rank(A) < i):
        A = np.random.randn(i, i)
    b = np.random.randn(i)

    if (np.mean(np.abs(np.linalg.solve(A, b) - solve_system(A, b))) > eps):
        print("bad system solver")

    if (np.mean(np.abs(np.linalg.lstsq(A, b, rcond=None)[0] - solve_lstsq(A, b))) > eps):
        print(f"bad lstsq solver, i={i}, diff={np.mean(np.abs(np.linalg.lstsq(A, b, rcond=None)[0] - solve_lstsq(A, b)))}")


bad lstsq solver, i=47, diff=0.00012120944744563072
